In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from skimage import img_as_uint
from skimage.transform import rotate
from imageio import imread
from tqdm import tqdm
from skimage import color as cl
from skimage.util import view_as_blocks
from skimage.transform import resize
from scipy.stats import mode
from sklearn import mixture

In [6]:
rawDF = pd.read_pickle('raw_food_df')
rawDF['SKImage'] = [img_as_uint(x.astype(int)) for x in rawDF['ImageArray']]
rawDF = rawDF.drop(columns = ['ImageArray'],axis=1)
rawDF = rawDF.drop(columns = ['ImageArray_nump'],axis=1)
prodsToDel = rawDF['ProdName'].value_counts()[rawDF['ProdName'].value_counts()<15].index
rawDF = rawDF.drop(rawDF[rawDF['ProdName'].isin(list(prodsToDel))].index,axis=0)
rawDF.describe()

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:126: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint16.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 255 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 146 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 152 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmle

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 253 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 246 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 241 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 250 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\env

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 171 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 196 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 228 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\dtype.py:179: UserWarning: Downcasting int32 to uint16 without scaling because max value 233 fits in uint16
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\AdamT\Anaconda3\env

,FullSize2D,Height,Width,Ratio
count,5978.000000,5978.000000,5978.000000,5978.000000
mean,29161.606223,147.836567,195.148879,1.431063
std,9175.752750,41.575850,23.580403,0.207731
min,304.000000,16.000000,19.000000,1.000000
25%,22600.000000,113.000000,200.000000,1.333333
50%,30000.000000,150.000000,200.000000,1.333333
75%,30000.000000,150.000000,200.000000,1.386476
max,139502.000000,373.000000,374.000000,3.014286


In [2]:
def rotateIm(img,angle):
    return rotate(img,angle,resize=True,preserve_range=True).astype(int)
def mirrorIm(img,axis=1):
    if(axis==0):
        return img[::-1,:,:]
    if(axis==1):
        return img[:,::-1,:]

def rotateChunk(df,angle):
    return df.SKImage.apply(lambda x: rotateIm(x,angle))
def mirrorChunk(df,ax):
    return df.SKImage.apply(lambda x: mirrorIm(x,ax))

In [12]:
targetAmount = 120
howMany = (targetAmount - rawDF['ProdName'].value_counts())
for prod in rawDF['ProdName'].value_counts()[howMany>0].index:
    amount = rawDF['ProdName'].value_counts()[prod]
    rest = targetAmount%amount
    mainfact = (targetAmount-rest)/amount
    print(prod, "; ", howMany[prod], "; ", mainfact, "*", amount, "+", rest)
    cnt=mainfact
    if mainfact>1:
        tempdf = rawDF[rawDF['ProdName'] == prod]
        tempdf.SKImage = mirrorChunk(tempdf[:],1)
        rawDF = rawDF.append(tempdf, ignore_index=True)
        cnt=cnt-1
    if mainfact>2:
        angles = [90,180,270,45]
        modu = mainfact%2
        print(modu)
        for i in range(int(((mainfact-modu)/2)-1+modu)):
            tempdf = rawDF[rawDF['ProdName'] == prod]
            tempdf.SKImage = rotateChunk(tempdf[:],angles[i])
            rawDF = rawDF.append(tempdf, ignore_index=True)
    if rest>0:
        tempdf = rawDF[rawDF['ProdName'] == prod]
        tempdf = tempdf.sample(n=rest)
        tempdf.SKImage = mirrorChunk(tempdf[:],0)
        print(tempdf.shape)
        rawDF = rawDF.append(tempdf, ignore_index=True)
rawDF['ProdName'].value_counts()

BUN ;  30 ;  1.0 * 90 + 30
(30, 9)
LITCHI ;  42 ;  1.0 * 78 + 42
(42, 9)
PEPPER ;  59 ;  1.0 * 61 + 59
(59, 9)
GRAPEFRUIT ;  67 ;  2.0 * 53 + 14


C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


(14, 9)
CARROT ;  74 ;  2.0 * 46 + 28
(28, 9)
WATERMELON ;  77 ;  2.0 * 43 + 34
(34, 9)
CUCUMBER ;  80 ;  3.0 * 40 + 0
1.0
OLIVE ;  82 ;  3.0 * 38 + 6
1.0
(6, 9)
APRICOT ;  89 ;  3.0 * 31 + 27
1.0
(27, 9)
POMEGRANATE ;  89 ;  3.0 * 31 + 27
1.0
(27, 9)
PAPAYA ;  91 ;  4.0 * 29 + 4
0.0
(4, 9)
AVOCADO ;  96 ;  5.0 * 24 + 0
1.0
KIWI ;  100 ;  6.0 * 20 + 0
0.0
DATES ;  103 ;  7.0 * 17 + 1
1.0
(1, 9)
AUBERGINE ;  105 ;  8.0 * 15 + 0
0.0
SWEET ;  105 ;  8.0 * 15 + 0
0.0


APPLE          593
ORANGE         516
ONION          341
EGG            335
TOMATO         332
BREAD          287
DATES          273
CHEESE         244
AUBERGINE      240
SWEET          240
MANGO          225
BEAN           224
SAUCE          210
DOUGHNUT       210
BANANA         206
PASTA          198
GRAPE          197
AVOCADO        192
LEMON          184
PEAR           178
PLUM           176
CUCUMBER       160
KIWI           160
OLIVE          158
APRICOT        151
POMEGRANATE    151
QIWI           150
SACHIMA        150
MOONCAKE       134
PEACH          133
FIRED          124
BUN            120
LITCHI         120
GRAPEFRUIT     120
PAPAYA         120
PEPPER         120
CARROT         120
WATERMELON     120
Name: ProdName, dtype: int64

In [3]:
def getHists(x):
    hists = []
    for i in range(3):
        hists.append(np.histogram(x[:,:,i],bins=50, density = True))
    return hists
def genHistArrays(df,csname):
    allpixV = np.zeros((df.shape[0],150))
    hists = df['SKImage'].apply(getHists)
    colrang = range(1,np.size(hists[0][0][1]))
    fullnames = []
    for chs in ['CH1', 'CH2', 'CH3']:
        fullnames.extend([chs+'-'+str(j) for j in colrang])
    fullnames = [csname+'-'+str(j) for j in fullnames]
    for rowi, pArr in enumerate(hists):
        pixVals = pArr[0][0]
        pixVals = np.append(pixVals,pArr[1][0])
        pixVals = np.append(pixVals,pArr[2][0])
        allpixV[rowi,:] = pixVals
        pixVals = None
    return allpixV,fullnames

In [13]:
def blocksForImg(img,samplsiz,resc):
    imgr = resize(img,(resc,resc,3))
    y=imgr.shape[0]
    x=imgr.shape[1]
    view = []
    view.append(view_as_blocks(imgr[:,:,0], (samplsiz,samplsiz)))
    view.append(view_as_blocks(imgr[:,:,1], (samplsiz,samplsiz)))
    view.append(view_as_blocks(imgr[:,:,2], (samplsiz,samplsiz)))
    flv = [v.reshape(v.shape[0],v.shape[1],-1) for v in view]
    meanb = np.array([np.mean(f,axis=2) for f in flv])
    #cmode = [mode(f,axis=2) for f in flv]
    stdb = np.array([np.std(f,axis=2) for f in flv])
    meanb = meanb.flatten()
    stdb = stdb.flatten()
    meanbsort = np.sort(meanb)
    stdbsort = np.sort(stdb)
    return np.append(meanb,[stdb,meanbsort,stdbsort])
    #return meanb.append(stdb.append(meanbsort.append(stdbsort)))

def blocksVals(df,ssiz,resc,cspacename='RGB'):
    amountBlocks = np.power(resc/ssiz,2)
    amountBlocks = int(amountBlocks)
    labelnames = []
    for ch in ['CH1','CH2','CH3']:   
        labelnames = np.append(labelnames,[cspacename+'-Mean-Block: ' + ch + ' ' + str(i) for i in range(1,amountBlocks+1)])
        labelnames = np.append(labelnames,[cspacename+'-Std-Block: ' + ch + ' ' + str(i) for i in range(1,amountBlocks+1)])
        labelnames = np.append(labelnames,[cspacename+'-Mean_Sorted-Block: ' + ch + ' ' + str(i) for i in range(1,amountBlocks+1)])
        labelnames = np.append(labelnames,[cspacename+'-Std_Sorted-Block: ' + ch + ' ' + str(i) for i in range(1,amountBlocks+1)])    
    blockArr = np.zeros((df.shape[0],np.size(labelnames)))
    blockDesc = df['SKImage'].apply(lambda x: blocksForImg(x,ssiz,resc))
    for i,bl in enumerate(blockDesc):
        blockArr[i,:] = bl
    #print(blockArr)
    return blockArr,labelnames

In [17]:
def runGMM(Xseg,n_kernels):
    gmm = mixture.GaussianMixture(n_components=n_kernels, covariance_type='full')
    gmm.fit(Xseg)
    labels = gmm.predict(Xseg)
    return labels
def prepareImg2GaussM(img):
    imgforSeg = np.zeros((img.shape[0]*img.shape[1],5))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            imgforSeg[(img.shape[0]*j)+i,:] = np.append(img[i][j],[j,i])
    Xseg = np.array(imgforSeg)
    return Xseg


def getClustersInfo(img,Xseg,labels,n_kernels):
    labsDat = pd.DataFrame(data = Xseg)
    labsDat['label'] = labels
    labsDat[3] = labsDat[3]/img.shape[0]
    labsDat[4] = labsDat[4]/img.shape[1]
    siz = np.zeros(n_kernels)
    mean = np.zeros((n_kernels,5))
    for i in range(5):
        siz[i] = labsDat.loc[labsDat['label']==i,0].shape[0]/labsDat.shape[0]
        mean[i,:] = labsDat.loc[labsDat['label']==i].mean(axis=0)[:-1]
    centrsDF = pd.DataFrame(data = mean)
    centrsDF['size'] = siz
    centrsDF = centrsDF.sort_values(by=0).append(centrsDF.sort_values(by=1)).append(
    centrsDF.sort_values(by=2)).append(centrsDF.sort_values(by=3)).append(
    centrsDF.sort_values(by=4)).append(centrsDF.sort_values(by='size'))
    centrDataArr = centrsDF.values.flatten()
    return centrDataArr

def prepareLabels(n_kernels,cspace):
    cantrDataLabels = []
    for i in range(n_kernels):
        for s in ['sortCH1','sortCH2','sortCH3','sortX','sortY','sortSize']:
            cantrDataLabels = np.append(
                cantrDataLabels, ['Cluster: ' + str(i)+' cspace: ' + cspace + '; ' + s + '; ' + var for var in
                             ['CH1', 'CH2', 'CH3', 'X', 'Y', 'SIZE']])
    return cantrDataLabels

def processImageGM(img, n_kernels, scale_factor = 1):
    if scale_factor != 1:
        img = resize(img,(int(img.shape[0]/scale_factor),int(img.shape[1]/scale_factor),3))
    data2seg = prepareImg2GaussM(img)
    labels = runGMM(data2seg,n_kernels)
    return getClustersInfo(img,data2seg,labels,n_kernels)
def getSegmentInfo(df,cspace,n_kernels, scale_factor = 1):
    colNames = prepareLabels(n_kernels,cspace)
    tqdm.pandas(ncols=50)
    segArr = np.zeros((df.shape[0],np.size(colNames)))
    allImgsSegInfo = df.SKImage.progress_apply(lambda x: processImageGM(x,n_kernels,scale_factor))
    for i,seg in enumerate(allImgsSegInfo):
        segArr[i,:] = seg
    #print(segArr)
    return segArr,colNames

In [6]:
def describeImgs(df,cspace,blocksResiz=160,blocksblSiz=20,n_clusters=5,clustResiz=3):
    histV,histCols = genHistArrays(df,cspace)
    histDF = pd.DataFrame(data = histV, columns = histCols)
    blocksV, blocksCols = blocksVals(df,blocksblSiz,blocksResiz, cspace)
    blocksDF = pd.DataFrame(data = blocksV, columns = blocksCols)
    segV, segCols = getSegmentInfo(df,cspace,n_clusters, clustResiz)
    segDF = pd.DataFrame(data = segV, columns = segCols)
    return pd.concat([histDF,blocksDF,segDF], axis=1)
def extractVarsForML(df,blocksResiz=160,blocksblSiz=20,n_clusters=5,clustResiz=3):
    resDF = describeImgs(df,'RGB',blocksResiz,blocksblSiz,n_clusters,clustResiz)
    cspdict = {'HSV': cl.rgb2hsv, 'CIE': cl.rgb2rgbcie, 'YDBDR': cl.rgb2ydbdr}
    for csp, colFunc in cspdict.items():
        df['SKImage'] = df['SKImage'].apply(colFunc)
        resDF = pd.concat([resDF, describeImgs(df,csp,blocksResiz,blocksblSiz,n_clusters,clustResiz)], axis=1)
    return resDF

In [8]:
rawDF = pd.read_pickle('rawDF2')
rawDF.describe()

,FullSize2D,Height,Width,Ratio
count,7912.000000,7912.000000,7912.000000,7912.000000
mean,26860.005182,140.309024,185.396234,1.451152
std,11185.869963,44.964427,41.084657,0.274865
min,304.000000,16.000000,19.000000,1.000000
25%,22400.000000,112.000000,200.000000,1.333333
50%,30000.000000,150.000000,200.000000,1.333333
75%,30000.000000,150.000000,200.000000,1.747573
max,139502.000000,373.000000,374.000000,3.014286


In [9]:
rawDFsl = rawDF[:5]

In [18]:
THE_PREPROCESSED_ONE = extractVarsForML(rawDF)

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\skimage\util\shape.py:94: RuntimeWarning: Cannot provide views on a non-contiguous input array without copying.
  warn(RuntimeWarning("Cannot provide views on a non-contiguous input "
 33%|██▉      | 2615/7912 [04:58<11:54,  7.42it/s]C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameter

In [19]:
THE_PREPROCESSED_ONE.head()

,RGB-CH1-1,RGB-CH1-2,RGB-CH1-3,RGB-CH1-4,RGB-CH1-5,RGB-CH1-6,RGB-CH1-7,RGB-CH1-8,RGB-CH1-9,RGB-CH1-10,...,Cluster: 4 cspace: YDBDR; sortY; CH3,Cluster: 4 cspace: YDBDR; sortY; X,Cluster: 4 cspace: YDBDR; sortY; Y,Cluster: 4 cspace: YDBDR; sortY; SIZE,Cluster: 4 cspace: YDBDR; sortSize; CH1,Cluster: 4 cspace: YDBDR; sortSize; CH2,Cluster: 4 cspace: YDBDR; sortSize; CH3,Cluster: 4 cspace: YDBDR; sortSize; X,Cluster: 4 cspace: YDBDR; sortSize; Y,Cluster: 4 cspace: YDBDR; sortSize; SIZE
0,1.769172,1.818316,0.835442,1.130304,0.540580,0.442293,2.211465,5.946384,11.155613,20.099760,...,0.127627,1.041432,0.418326,0.155201,0.245593,-0.237904,0.150231,0.960264,0.229676,0.591319
1,0.111782,0.000000,0.000000,0.000000,0.037261,0.260824,6.408822,10.693789,6.408822,16.171096,...,0.150522,1.038269,0.502485,0.266061,0.246006,-0.238255,0.150847,0.403593,0.444197,0.315455
2,0.881083,0.713258,0.923039,2.097817,10.992560,10.782778,12.712770,18.125137,97.926087,75.101840,...,0.150444,0.888104,0.614620,0.233333,0.245732,-0.238081,0.150439,0.554374,0.303041,0.442727
3,0.470420,0.108558,0.036186,0.072372,0.036186,0.036186,0.217117,0.832281,1.157956,4.089033,...,0.134658,0.376533,0.577010,0.113636,0.245824,-0.238045,0.150616,0.712736,0.380198,0.703333
4,0.166011,0.456530,1.701613,2.822188,5.187845,32.911689,85.703199,55.489190,53.912085,36.273412,...,0.119049,0.232096,0.135404,0.138788,0.245744,-0.238173,0.150429,0.597301,0.425353,0.595152


In [20]:
THE_PREPROCESSED_ONE.to_pickle('THE_PREPROCESSED_ONE')

In [22]:
rawDF.to_pickle('rawDF2')

In [2]:
THE_PREPROCESSED_ONE = pd.read_pickle('THE_PREPROCESSED_ONE')
THE_PREPROCESSED_ONE.describe()

,RGB-CH1-1,RGB-CH1-2,RGB-CH1-3,RGB-CH1-4,RGB-CH1-5,RGB-CH1-6,RGB-CH1-7,RGB-CH1-8,RGB-CH1-9,RGB-CH1-10,...,Kernel: 4 cspace: YDBDR; sortY; CH3,Kernel: 4 cspace: YDBDR; sortY; X,Kernel: 4 cspace: YDBDR; sortY; Y,Kernel: 4 cspace: YDBDR; sortY; SIZE,Kernel: 4 cspace: YDBDR; sortSize; CH1,Kernel: 4 cspace: YDBDR; sortSize; CH2,Kernel: 4 cspace: YDBDR; sortSize; CH3,Kernel: 4 cspace: YDBDR; sortSize; X,Kernel: 4 cspace: YDBDR; sortSize; Y,Kernel: 4 cspace: YDBDR; sortSize; SIZE
count,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000,...,7912.000000,7912.000000,7912.000000,7912.000000,7.912000e+03,7912.000000,7912.000000,7912.000000,7912.000000,7912.000000
mean,0.000732,0.000796,0.001387,0.001946,0.002317,0.002690,0.002923,0.002959,0.002972,0.002949,...,0.017185,0.633502,0.414875,0.251369,4.587830e-02,-0.056347,0.024642,0.636456,0.418542,0.386704
std,0.002419,0.002247,0.003176,0.003987,0.004285,0.004635,0.004885,0.004331,0.003976,0.003647,...,0.051302,0.368071,0.263398,0.056195,2.366811e-02,0.028946,0.051916,0.314802,0.226272,0.105001
min,0.000003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.141661,0.033833,0.021323,0.054464,2.919394e-09,-0.178106,-0.127054,0.075290,0.086543,0.200000
25%,0.000026,0.000030,0.000063,0.000105,0.000161,0.000222,0.000303,0.000380,0.000495,0.000592,...,-0.014320,0.336349,0.219375,0.219697,2.647419e-02,-0.075237,-0.003134,0.388050,0.265324,0.314087
50%,0.000070,0.000132,0.000239,0.000371,0.000541,0.000743,0.001043,0.001282,0.001528,0.001702,...,0.027043,0.579708,0.362127,0.251212,4.542198e-02,-0.055694,0.035888,0.599145,0.367726,0.360770
75%,0.000239,0.000522,0.001060,0.001636,0.002235,0.002942,0.003456,0.003605,0.003781,0.003828,...,0.057578,0.873455,0.554094,0.287879,6.192500e-02,-0.032728,0.064633,0.815991,0.524640,0.431206
max,0.076116,0.030364,0.036558,0.029615,0.037556,0.045096,0.044907,0.043131,0.038441,0.039628,...,0.128038,2.442138,2.439819,0.443333,1.406451e-01,0.000847,0.124561,2.167011,2.167304,0.827664


In [44]:
THE_PREPROCESSED_ONE.isnull().sum(axis=0).sort_values()

RGB-CH1-1                                  0
CIE-Mean-Block: 61                         0
CIE-Mean-Block: 62                         0
CIE-Mean-Block: 63                         0
CIE-Mean-Block: 64                         0
CIE-Std-Block: 1                           0
CIE-Std-Block: 2                           0
CIE-Std-Block: 3                           0
CIE-Std-Block: 4                           0
CIE-Std-Block: 5                           0
CIE-Std-Block: 6                           0
CIE-Std-Block: 7                           0
CIE-Std-Block: 8                           0
CIE-Std-Block: 9                           0
CIE-Std-Block: 10                          0
CIE-Mean-Block: 60                         0
CIE-Std-Block: 11                          0
CIE-Std-Block: 13                          0
CIE-Std-Block: 14                          0
CIE-Std-Block: 15                          0
CIE-Std-Block: 16                          0
CIE-Std-Block: 17                          0
CIE-Std-Bl

In [9]:
THE_PREPROCESSED_ONE.shape

(7912, 4392)

In [46]:
THE_PREPROCESSED_ONE2 = THE_PREPROCESSED_ONE.dropna(axis=0)
THE_PREPROCESSED_ONE2.shape

(7912, 4392)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
X = THE_PREPROCESSED_ONE
y = rawDF['ProdName']

In [14]:
y.shape

(7912,)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 666)#because hail satan
print("X train shape: ", X_train.shape)
print("y train shape: ", y_train.shape)
print("======================")
print("X test shape: ", X_test.shape)
print("y test shape: ", y_test.shape)

X train shape:  (6329, 4392)
y train shape:  (6329,)
X test shape:  (1583, 4392)
y test shape:  (1583,)


In [21]:
#pipeline = make_pipeline(StandardScaler(),SelectFromModel(RandomForestClassifier(100),threshold=0.05),
#                          RandomForestClassifier())

pipeline = make_pipeline(StandardScaler(),
                          RandomForestClassifier())

scs = cross_val_score(pipeline,scale(X_train),y_train,cv=4,verbose=1,n_jobs=-1)
print(scs.mean())
print(scs)

[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    7.6s remaining:    7.6s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.5s finished


0.8123245233678031
[0.80525986 0.82471627 0.79354022 0.82578175]


In [39]:
scs

NameError: name 'scs' is not defined